In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
DATASET = "./Dataset"
df_full = pd.read_csv(os.path.join(DATASET, "campeonato-brasileiro-full.csv"))
df_estatisticas_full = pd.read_csv(os.path.join(DATASET, "campeonato-brasileiro-estatisticas-full.csv"))
df_cartoes = pd.read_csv(os.path.join(DATASET, "campeonato-brasileiro-full.csv"))

# Eliminando as linhas sem todas as informações

In [3]:
start_id_complete = 4986
df_full_complete = df_full.iloc[start_id_complete:]
len(df_full_complete)

3039

### Eliminando as linhas com NaN restantes

In [4]:
nan_rows = df_full_complete[df_full_complete.isna().any(axis=1)]
print("Linhas com NaN: ", len(nan_rows))
df_full_complete = df_full_complete.dropna()
df_full_complete = df_full_complete.reset_index()
df_full_complete = df_full_complete.drop("index", axis=1)
print("Linhas totais:", len(df_full_complete))
ids_removed = list(nan_rows["ID"])

Linhas com NaN:  7
Linhas totais: 3032


# Adicionando dataset de estatisticas-full no dataset base

### Eliminando as linhas sem informações completas

In [5]:
print("Linhas totais:", len(df_estatisticas_full))
df_est_comp = df_estatisticas_full.iloc[2*start_id_complete:]
print("Linhas totais:", len(df_est_comp))

Linhas totais: 16050
Linhas totais: 6078


In [6]:
df_est_comp = df_est_comp.drop(index=df_est_comp[df_est_comp['partida_id'].isin(ids_removed)].index)
print("Linhas totais:", len(df_est_comp))

Linhas totais: 6064


### Colocando as informações do mandante e visitante em uma linha só

Adicionando colunas de mandante e visitante vazias

In [7]:
add_columns = [
    "chutes_mandante", 
    "chutes_no_alvo_mandante",
    "posse_de_bola_mandante",
    "passes_mandante",
    "precisao_passes_mandante",
    "faltas_mandante",
    "cartao_amarelo_mandante",
    "cartao_vermelho_mandante",
    "impedimentos_mandante",
    "escanteios_mandante",
    "chutes_visitante", 
    "chutes_no_alvo_visitante",
    "posse_de_bola_visitante",
    "passes_visitante",
    "precisao_passes_visitante",
    "faltas_visitante",
    "cartao_amarelo_visitante",
    "cartao_vermelho_visitante",
    "impedimentos_visitante",
    "escanteios_visitante",
    ]
null_values = [np.nan]*len(add_columns)
for new_col in add_columns:
    df_est_comp.loc[:, new_col] = np.nan

df_est_comp = df_est_comp.reset_index()
df_est_comp = df_est_comp.drop("index", axis=1)
df_est_comp.head()

,partida_id,rodata,clube,chutes,chutes_no_alvo,posse_de_bola,passes,precisao_passes,faltas,cartao_amarelo,...,chutes_visitante,chutes_no_alvo_visitante,posse_de_bola_visitante,passes_visitante,precisao_passes_visitante,faltas_visitante,cartao_amarelo_visitante,cartao_vermelho_visitante,impedimentos_visitante,escanteios_visitante
0,4987,1,Chapecoense,13,0,45%,326,NaN,20,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4987,1,Coritiba,16,0,55%,388,NaN,21,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4988,1,Palmeiras,17,0,50%,406,NaN,20,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4988,1,Atletico-MG,10,0,50%,426,NaN,14,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4989,1,Fluminense,26,11,74%,702,NaN,9,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print("Linhas com precisão de passes:", len(df_est_comp["precisao_passes"]) - df_est_comp["precisao_passes"].isna().sum())
print("Linhas sem precisão de passes:", df_est_comp["precisao_passes"].isna().sum())

Linhas com precisão de passes: 3752
Linhas sem precisão de passes: 2312


Preenchendo mandante e visitante em apenas uma linha, e apagando a restante 

In [9]:
def fill_mandante(row):
    return pd.Series({
        "chutes_mandante": row["chutes"],
        "chutes_no_alvo_mandante": row["chutes_no_alvo"],
        "posse_de_bola_mandante": row["posse_de_bola"],
        "passes_mandante": row["passes"],
        "precisao_passes_mandante": row["precisao_passes"],
        "faltas_mandante": row["faltas"],
        "cartao_amarelo_mandante": row["cartao_amarelo"],
        "cartao_vermelho_mandante": row["cartao_vermelho"],
        "impedimentos_mandante": row["impedimentos"],
        "escanteios_mandante": row["escanteios"],
    })

def fill_visitante(row):
    return pd.Series({
        "chutes_visitante": row["chutes"],
        "chutes_no_alvo_visitante": row["chutes_no_alvo"],
        "posse_de_bola_visitante": row["posse_de_bola"],
        "passes_visitante": row["passes"],
        "precisao_passes_visitante": row["precisao_passes"],
        "faltas_visitante": row["faltas"],
        "cartao_amarelo_visitante": row["cartao_amarelo"],
        "cartao_vermelho_visitante": row["cartao_vermelho"],
        "impedimentos_visitante": row["impedimentos"],
        "escanteios_visitante": row["escanteios"],
    })


In [10]:
cols_mandante = [
        "chutes_mandante", 
        "chutes_no_alvo_mandante",
        "posse_de_bola_mandante",
        "passes_mandante",
        "precisao_passes_mandante",
        "faltas_mandante",
        "cartao_amarelo_mandante",
        "cartao_vermelho_mandante",
        "impedimentos_mandante",
        "escanteios_mandante"
    ]

cols_visitante = [
    "chutes_visitante", 
    "chutes_no_alvo_visitante",
    "posse_de_bola_visitante",
    "passes_visitante",
    "precisao_passes_visitante",
    "faltas_visitante",
    "cartao_amarelo_visitante",
    "cartao_vermelho_visitante",
    "impedimentos_visitante",
    "escanteios_visitante",
]


len_est_comp = len(df_est_comp)
for i in range(0, len_est_comp, 2):
    clube = df_est_comp.loc[i, "clube"]
    id = df_est_comp.loc[i, "partida_id"]
    row_base = df_full_complete.loc[df_full_complete["ID"] == id]
    if row_base["mandante"].values[0] == clube:
        df_est_comp.loc[i, cols_mandante] = fill_mandante(df_est_comp.loc[i])
        df_est_comp.loc[i, cols_visitante] = fill_visitante(df_est_comp.loc[i+1])
    elif row_base["visitante"].values[0] == clube:
        df_est_comp.loc[i, cols_visitante] = fill_visitante(df_est_comp.loc[i])
        df_est_comp.loc[i, cols_mandante] = fill_mandante(df_est_comp.loc[i+1])
    else:
        print("Deu merda") # Não deu merda \o/
    df_est_comp = df_est_comp.drop(index=i+1)

df_est_comp = df_est_comp.reset_index()
df_est_comp = df_est_comp.drop("index", axis=1)

Eliminando colunas extras

In [11]:
df_est_comp = df_est_comp.drop([
    "rodata", 
    "clube", 
    "chutes", 
    "chutes_no_alvo", 
    "posse_de_bola", 
    "passes",
    "precisao_passes",
    "faltas",
    "cartao_amarelo",
    "cartao_vermelho",
    "impedimentos",
    "escanteios"
], 
    axis=1
)
df_est_comp = df_est_comp.rename(columns={"partida_id": "ID"})
df_est_comp

,ID,chutes_mandante,chutes_no_alvo_mandante,posse_de_bola_mandante,passes_mandante,precisao_passes_mandante,faltas_mandante,cartao_amarelo_mandante,cartao_vermelho_mandante,impedimentos_mandante,...,chutes_visitante,chutes_no_alvo_visitante,posse_de_bola_visitante,passes_visitante,precisao_passes_visitante,faltas_visitante,cartao_amarelo_visitante,cartao_vermelho_visitante,impedimentos_visitante,escanteios_visitante
0,4987,13.0,0.0,45%,326.0,NaN,20.0,2.0,0.0,3.0,...,16.0,0.0,55%,388.0,NaN,21.0,2.0,0.0,0.0,5.0
1,4988,17.0,0.0,50%,406.0,NaN,20.0,2.0,0.0,2.0,...,10.0,0.0,50%,426.0,NaN,14.0,4.0,0.0,4.0,2.0
2,4989,26.0,11.0,74%,702.0,NaN,9.0,2.0,0.0,1.0,...,3.0,0.0,26%,250.0,NaN,14.0,0.0,1.0,1.0,3.0
3,4990,11.0,0.0,45%,401.0,NaN,17.0,3.0,0.0,2.0,...,17.0,0.0,55%,473.0,NaN,21.0,4.0,1.0,0.0,10.0
4,4991,10.0,0.0,39%,335.0,NaN,17.0,1.0,0.0,1.0,...,17.0,0.0,61%,502.0,NaN,17.0,2.0,0.0,1.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3027,8021,17.0,8.0,57%,505.0,88%,9.0,1.0,0.0,2.0,...,4.0,2.0,43%,389.0,82%,8.0,3.0,1.0,2.0,1.0
3028,8022,13.0,4.0,41%,319.0,77%,16.0,3.0,0.0,5.0,...,8.0,4.0,59%,463.0,82%,15.0,2.0,0.0,0.0,9.0
3029,8023,17.0,4.0,59%,494.0,84%,11.0,3.0,0.0,1.0,...,17.0,7.0,41%,351.0,82%,19.0,6.0,0.0,0.0,4.0
3030,8024,15.0,8.0,48%,287.0,77%,13.0,2.0,0.0,1.0,...,11.0,2.0,52%,306.0,82%,17.0,3.0,0.0,2.0,5.0


In [12]:
df_full_complete

,ID,rodata,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado
0,4987,1,9/5/2015,18:32,Chapecoense,Coritiba,4-2-3-1,3-5-2,V. Eutrópio,M. dos Santos Gonçalves,Chapecoense,Arena Condá,2,1,SC,PR
1,4988,1,9/5/2015,18:33,Palmeiras,Atletico-MG,4-2-3-1,4-2-3-1,O. de Oliveira Filho,L. Culpi,-,Allianz Parque,2,2,SP,MG
2,4989,1,9/5/2015,21:00,Fluminense,Joinville,4-3-1-2,4-4-1-1,S. Drubscky de Campos,H. Maria,Fluminense,Maracanã,1,0,RJ,SC
3,4990,1,10/5/2015,11:00,Gremio,Ponte Preta,4-2-3-1,4-1-2-1-2,L. Scolari,A. Ferreira,-,Arena do Grêmio,3,3,RS,SP
4,4991,1,10/5/2015,16:00,Athletico-PR,Internacional,4-2-3-1,4-2-3-1,M. Mendes,D. Aguirre Camblor,Athletico-PR,Estádio Joaquim Américo Guimarães,3,0,PR,RS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3027,8021,38,13/11/2022,16:03,Cuiaba,Coritiba,4-1-4-1,4-1-4-1,A. Cardoso de Oliveira,A. Ferreira,Cuiaba,Arena Pantanal,2,1,MT,PR
3028,8022,38,13/11/2022,16:03,Bragantino,Fluminense,4-2-3-1,4-2-3-1,M. Nogueira Barbieri,F. Diniz Silva,Fluminense,Nabizão,0,1,SP,RJ
3029,8023,38,13/11/2022,16:03,Corinthians,Atletico-MG,4-1-4-1,4-2-3-1,F. J. Monteiro Almeida,A. Stival,Atletico-MG,Neo Química Arena,0,1,SP,MG
3030,8024,38,13/11/2022,16:03,Internacional,Palmeiras,4-2-3-1,4-2-3-1,L. Venker de Menezes,A. Moreira Ferreira,Internacional,Estádio José Pinheiro Borda,3,0,RS,SP


### Juntar com o dataset base

In [13]:
df_full_complete = df_full_complete.join(df_est_comp.set_index("ID"), on="ID")
df_full_complete

,ID,rodata,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,...,chutes_visitante,chutes_no_alvo_visitante,posse_de_bola_visitante,passes_visitante,precisao_passes_visitante,faltas_visitante,cartao_amarelo_visitante,cartao_vermelho_visitante,impedimentos_visitante,escanteios_visitante
0,4987,1,9/5/2015,18:32,Chapecoense,Coritiba,4-2-3-1,3-5-2,V. Eutrópio,M. dos Santos Gonçalves,...,16.0,0.0,55%,388.0,NaN,21.0,2.0,0.0,0.0,5.0
1,4988,1,9/5/2015,18:33,Palmeiras,Atletico-MG,4-2-3-1,4-2-3-1,O. de Oliveira Filho,L. Culpi,...,10.0,0.0,50%,426.0,NaN,14.0,4.0,0.0,4.0,2.0
2,4989,1,9/5/2015,21:00,Fluminense,Joinville,4-3-1-2,4-4-1-1,S. Drubscky de Campos,H. Maria,...,3.0,0.0,26%,250.0,NaN,14.0,0.0,1.0,1.0,3.0
3,4990,1,10/5/2015,11:00,Gremio,Ponte Preta,4-2-3-1,4-1-2-1-2,L. Scolari,A. Ferreira,...,17.0,0.0,55%,473.0,NaN,21.0,4.0,1.0,0.0,10.0
4,4991,1,10/5/2015,16:00,Athletico-PR,Internacional,4-2-3-1,4-2-3-1,M. Mendes,D. Aguirre Camblor,...,17.0,0.0,61%,502.0,NaN,17.0,2.0,0.0,1.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3027,8021,38,13/11/2022,16:03,Cuiaba,Coritiba,4-1-4-1,4-1-4-1,A. Cardoso de Oliveira,A. Ferreira,...,4.0,2.0,43%,389.0,82%,8.0,3.0,1.0,2.0,1.0
3028,8022,38,13/11/2022,16:03,Bragantino,Fluminense,4-2-3-1,4-2-3-1,M. Nogueira Barbieri,F. Diniz Silva,...,8.0,4.0,59%,463.0,82%,15.0,2.0,0.0,0.0,9.0
3029,8023,38,13/11/2022,16:03,Corinthians,Atletico-MG,4-1-4-1,4-2-3-1,F. J. Monteiro Almeida,A. Stival,...,17.0,7.0,41%,351.0,82%,19.0,6.0,0.0,0.0,4.0
3030,8024,38,13/11/2022,16:03,Internacional,Palmeiras,4-2-3-1,4-2-3-1,L. Venker de Menezes,A. Moreira Ferreira,...,11.0,2.0,52%,306.0,82%,17.0,3.0,0.0,2.0,5.0


Restaram apenas 24 linhas com NAN e são na posse_de_bola, tanto mandante qnt visitante

In [14]:
df_full_complete.to_csv(os.path.join(DATASET, "dataset_completo_e_tratado.csv"))